### Notebook adding Additional Features

In [1]:
#Import Libraries
import pandas as pd
import numpy as np

from pandas.tseries.offsets import MonthEnd

In [2]:
csv_data = pd.read_csv('SP500_Index_Data_partial_clean.csv')

csv_data['Date'] = pd.to_datetime(csv_data['Date'], format="%Y-%m-%d") #Date column to datetime
csv_data = csv_data.sort_values(by='Date', ascending=True).reset_index(drop=True) #dates sorted to better visualize output
csv_data.drop(['USD_FF_mktcap'],axis=1,inplace=True) #remove unwanted columns
csv_data.rename(columns={'Local_Returns_12m': 'Price_Returns_12m', 'Local_Returns_1m': 'Price_Returns_1m'}, inplace=True)  #rename certain columns


export_df = csv_data.copy() #copy of starting data, used later when we export the new features

csv_data

,Date,Ticker,Name,Sector,Price,Price_Returns_12m,Price_Returns_1m,Trail_DivYld,PB,Trail_EV_EBITDA,Trail_PE,Trail_EPS,Trail3yrAvg_EPSgro,Trail3yrAvg_DPSgro,Volatility,Debt_to_MktCap,NetDebt_EBITDA
0,2000-12-31,VTSS,MICROCHIP COMMUN,Information Technology,1106.2500,0.054827,0.282609,0.0000,8.6338,36.6065,80.1630,13.8000,66.4983,0.0000,114.207,0.0721,-0.1106
1,2000-12-31,RX,IQVIA INC,Information Technology,27.0000,0.020148,-0.035714,0.2963,75.9729,18.0078,31.3953,0.8600,NaN,0.0000,56.091,0.0485,0.5787
2,2000-12-31,MMC,MARSH & MCLENNAN,Financials,58.5000,0.245283,0.016287,1.6239,6.1773,12.6324,28.5366,2.0500,-0.0940,14.4714,46.930,0.0834,0.8887
3,2000-12-31,MAS,MASCO CORP,Industrials,22.5765,0.037589,0.330097,2.1704,2.9307,NaN,14.8530,1.5200,2.1774,6.5566,50.488,0.2817,NaN
4,2000-12-31,BNI,BURLINGTON NORTH,Industrials,28.3125,0.191993,0.123669,1.6954,1.5223,NaN,11.9462,2.3700,4.1158,6.2659,44.893,0.5997,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123863,2021-06-30,ROK,ROCKWELL AUTOMAT,Industrials,286.0200,0.365998,0.084559,1.4789,18.2737,24.9142,34.3342,8.3305,8.7257,8.0849,25.230,0.0715,1.3119
123864,2021-06-30,KHC,KRAFT HEINZ CO/T,Consumer Staples,40.7800,0.335186,-0.064464,3.9235,0.9918,21.7808,13.5238,3.0154,-2.9751,-13.8226,24.890,0.5454,7.3184
123865,2021-06-30,AMT,AMERICAN TOWER C,Real Estate,270.1400,0.066064,0.062448,1.7991,30.9410,24.0730,60.6925,4.4510,34.7038,19.0551,23.760,0.2987,5.9055
123866,2021-06-30,DVA,DAVITA INC,Health Care,120.4300,0.521734,0.002998,0.0000,10.3320,10.4995,14.6562,8.2170,35.1035,0.0000,27.763,0.9469,4.8479


## Imputation of Trailing 1 Year EPS Growth (Trail1yr_EPSgro)

In [3]:
df = csv_data.copy()

#df = df[df.Ticker == 'IBM']  #Sanity check using one stock

### Limit to just the columns we need, date ticker and trailing 1 yr EPS
df = df[['Date', 'Ticker','Trail_EPS']]

df = df.sort_values(by='Date', ascending=True) #dates sorted to better visualize output

df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d") #Date column to datetime

### This pandas method provides the monthly date we want from 12 months ago in a new column
df['Date_1year_Ago'] = df['Date'] + MonthEnd(-12)

### Temp df on which we'll 'reverse merge' the year prior EPS data
### this merge 'tricks' pandas to give us the year ago EPS
temp_df = df.copy()[['Date_1year_Ago', 'Ticker']] #copy of df with just the columns we need
temp_df.columns = ['Date', 'Ticker'] #change the column names to match df
temp_df = temp_df.merge(df[['Date','Ticker','Trail_EPS']], how='left', on=['Date', 'Ticker']) #this merge 'tricks' pandas to give us the year ago EPS
temp_df['Date_1year_Ago'] = temp_df['Date'] #flip column names to continue 'tricking' pandas
temp_df['Date'] = temp_df['Date_1year_Ago'] + MonthEnd(12) #make our 'Date' one year hence to continue the 'trick'
temp_df.columns = ['Date', 'Ticker', 'Trail_EPS_1year_Ago', 'Date_1year_Ago'] #rename the temp_df columns so we merge with the existing data properly
temp_df = temp_df[['Date', 'Ticker', 'Date_1year_Ago','Trail_EPS_1year_Ago']] #re-orders the columns

### apply the 'trick', merging df and temp_df with 1 year ago EPS
df = df.merge(temp_df, how='left', on=['Date', 'Ticker', 'Date_1year_Ago'])
del temp_df #no longer need temp_df

### calculate the growth rate
### Note that we multiply by 100 to match the formatting of our other growth rates (i.e. a '5' is 5%, a 0.05 is 0.05%)
df['Trail1yr_EPSgro'] = 100* ( df['Trail_EPS'] / df['Trail_EPS_1year_Ago'] -1 )

### If both prior and current EPS is zero (which seems unlikely, but is possible), correct the calculation to reflect 0% instead of NaN
df['Trail1yr_EPSgro'] = df.apply(lambda x: 0 if ((x['Trail_EPS']==0) & (x['Trail_EPS_1year_Ago']==0)) else 
                                        x['Trail1yr_EPSgro'], 
                                        axis=1)

### Get rid of inf and -inf values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

### Check the output
display(df.describe())

display(df.tail(24))

,Trail_EPS,Trail_EPS_1year_Ago,Trail1yr_EPSgro
count,116786.000000,105850.000000,104887.000000
mean,3.257297,3.206245,6.718690
std,8.558379,8.362580,1461.089931
min,-562.800000,-562.800000,-108594.285714
25%,1.240000,1.250000,-9.981677
50%,2.320000,2.293300,7.925311
75%,4.024100,3.941500,22.699505
max,287.765200,221.471100,221250.000000


,Date,Ticker,Trail_EPS,Date_1year_Ago,Trail_EPS_1year_Ago,Trail1yr_EPSgro
123844,2021-06-30,AWK,3.9600,2020-06-30,3.9185,1.059079
123845,2021-06-30,MMM,9.5480,2020-06-30,8.6898,9.875947
123846,2021-06-30,MRK,5.5477,2020-06-30,5.5362,0.207724
123847,2021-06-30,MCD,6.5247,2020-06-30,6.3737,2.369111
123848,2021-06-30,JNJ,7.6233,2020-06-30,6.7311,13.254891
123849,2021-06-30,IBM,9.4694,2020-06-30,11.4870,-17.564203
123850,2021-06-30,MPWR,3.7886,2020-06-30,NaN,NaN
123851,2021-06-30,TSN,6.5985,2020-06-30,4.9298,33.849243
123852,2021-06-30,HD,13.7906,2020-06-30,10.0600,37.083499
123853,2021-06-30,PSX,-2.8494,2020-06-30,3.4869,-181.717285


In [4]:
### merge new field with starting csv data so we can export
df = df[['Date','Ticker','Trail1yr_EPSgro']] #limit to just the columns we need to merge
#export_df = csv_data.copy() #copy of starting data
export_df = export_df.merge(df, how='left', on=['Date','Ticker'])
export_df.drop(['Trail_EPS'], axis=1, inplace=True) #drop the Trail_EPS column as we no longer need it, nor is it a useable feature

#export_df

## Imputation of Trailing 1 Year Dividend Per Share Growth (Trail1yr_DPSgro)

In [5]:
df = csv_data.copy()

### Impute the Trailing dividend per share or Trail_DPS field
### This is done by taking the dividend yield * the stock price
df['Trail_DPS'] = (df['Trail_DivYld'] / 100) * df['Price'] #dividend yield * the stock price is the imputed trailing DPS

#df = df[df.Ticker == 'IBM']  #Sanity check using one stock

### Limit to just the columns we need, date ticker and trailing 1 yr DPS
df = df[['Date', 'Ticker','Trail_DPS']]

df = df.sort_values(by='Date', ascending=True) #dates sorted to better visualize output

df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d") #Date column to datetime

### This pandas method provides the monthly date we want from 12 months ago in a new column
df['Date_1year_Ago'] = df['Date'] + MonthEnd(-12)

### Temp df on which we'll 'reverse merge' the year prior DPS data
### this merge 'tricks' pandas to give us the year ago DPS
temp_df = df.copy()[['Date_1year_Ago', 'Ticker']] #copy of df with just the columns we need
temp_df.columns = ['Date', 'Ticker'] #change the column names to match df

temp_df = temp_df.merge(df[['Date','Ticker','Trail_DPS']], how='left', on=['Date', 'Ticker']) #this merge 'tricks' pandas to give us the year ago EPS
temp_df['Date_1year_Ago'] = temp_df['Date'] #flip column names to continue 'tricking' pandas
temp_df['Date'] = temp_df['Date_1year_Ago'] + MonthEnd(12) #make our 'Date' one year hence to continue the 'trick'
temp_df.columns = ['Date', 'Ticker', 'Trail_DPS_1year_Ago', 'Date_1year_Ago'] #rename the temp_df columns so we merge with the existing data properly
temp_df = temp_df[['Date', 'Ticker', 'Date_1year_Ago','Trail_DPS_1year_Ago']] #re-orders the columns
#display(temp_df)

### apply the 'trick', merging df and temp_df with 1 year ago EPS
df = df.merge(temp_df, how='left', on=['Date', 'Ticker', 'Date_1year_Ago'])
del temp_df #no longer need temp_df

### calculate the growth rate
### Note that we multiply by 100 to match the formatting of our other growth rates (i.e. a '5' is 5%, a 0.05 is 0.05%)
df['Trail1yr_DPSgro'] = 100* ( df['Trail_DPS'] / df['Trail_DPS_1year_Ago'] -1 )

### If both prior and current dividend is zero, correct the calculation to reflect 0% instead of NaN
df['Trail1yr_DPSgro'] = df.apply(lambda x: 0 if ((x['Trail_DPS']==0) & (x['Trail_DPS_1year_Ago']==0)) else 
                                        x['Trail1yr_DPSgro'], 
                                        axis=1)

### Get rid of inf and -inf values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

### Check the output
display(df.describe())

#display(df.head(24))
 
display(df.tail(24))

,Trail_DPS,Trail_DPS_1year_Ago,Trail1yr_DPSgro
count,119087.000000,108022.000000,106120.000000
mean,1.032322,0.997162,15.535515
std,1.783054,1.685477,149.765204
min,0.000000,0.000000,-100.000000
25%,0.100000,0.100003,0.000000
50%,0.649983,0.639985,4.350157
75%,1.400010,1.360014,13.158668
max,106.070009,62.500127,9915.485168


,Date,Ticker,Trail_DPS,Date_1year_Ago,Trail_DPS_1year_Ago,Trail1yr_DPSgro
123844,2021-06-30,AWK,2.252456,2020-06-30,2.049940,9.879121
123845,2021-06-30,MMM,5.899907,2020-06-30,5.819987,1.373199
123846,2021-06-30,MRK,2.560033,2020-06-30,2.379983,7.565159
123847,2021-06-30,MCD,5.119893,2020-06-30,4.910038,4.274007
123848,2021-06-30,JNJ,4.090000,2020-06-30,3.860012,5.958213
123849,2021-06-30,IBM,6.529998,2020-06-30,6.490059,0.615389
123850,2021-06-30,MPWR,2.199994,2020-06-30,NaN,NaN
123851,2021-06-30,TSN,1.754972,2020-06-30,1.634979,7.339082
123852,2021-06-30,HD,6.299991,2020-06-30,5.719895,10.141725
123853,2021-06-30,PSX,3.599977,2020-06-30,3.600033,-0.001546


In [6]:
### merge new field with starting csv data so we can export
df = df[['Date','Ticker','Trail1yr_DPSgro']] #limit to just the columns we need to merge
#export_df = csv_data.copy() #copy of starting data
export_df = export_df.merge(df, how='left', on=['Date','Ticker'])

#export_df

## Imputation of Trailing 12 month Price Performance using 1 month Performance 

###(To Test our Method -- Matches to our existing 12m performance data)

In [7]:
# df = csv_data.copy()

# df = df[df.Ticker == 'IBM']  #Sanity check using one stock


# ### Limit to just the columns we need
# df = df[['Date', 'Ticker','Price_Returns_1m']]

# #display(df)


# ### Shift by __ months and calculate performance consistent with the existing performance columns
# ### Need the '+1' for 'chaining' the performance together for an accurate performance calc.
# df['1m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(0) +1
# df['2m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(1) +1
# df['3m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(2) +1
# df['4m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(3) +1
# df['5m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(4) +1
# df['6m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(5) +1
# df['7m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(6) +1
# df['8m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(7) +1
# df['9m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(8) +1
# df['10m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(9) +1
# df['11m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(10) +1
# df['12m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(11) +1

# ### Perform the chained performance calc
# df['Price_Returns_12m_ALT'] = ( df['1m']*df['2m']*df['3m']*df['4m']*df['5m']*df['6m']*df['7m']*df['8m']*df['9m']*df['10m']*df['11m']*df['12m'] )-1

# ### Check the output
# display(df.describe())

# #display(df.head(24))
 
# display(df.tail(24))

## Imputation of Trailing 3 month Price Performance

In [8]:
df = csv_data.copy()

#df = df[df.Ticker == 'IBM']  #Sanity check using one stock


### Limit to just the columns we need
df = df[['Date', 'Ticker','Price_Returns_1m']]

#display(df)


### Shift by __ months and calculate performance consistent with the existing performance columns
### Need the '+1' for 'chaining' the performance together for an accurate performance calc.
df['1m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(0) +1
df['2m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(1) +1
df['3m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(2) +1
# df['4m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(3) +1
# df['5m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(4) +1
# df['6m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(5) +1
# df['7m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(6) +1
# df['8m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(7) +1
# df['9m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(8) +1
# df['10m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(9) +1
# df['11m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(10) +1
# df['12m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(11) +1

### Perform the chained performance calc
df['Price_Returns_3m'] = ( df['1m']*df['2m']*df['3m'] )-1 #*df['4m']*df['5m']*df['6m']*df['7m']*df['8m']*df['9m']*df['10m']*df['11m']*df['12m'] )-1

### Check the output
display(df.describe())

#display(df.head(24))
 
display(df.tail(24))

,Price_Returns_1m,1m,2m,3m,Price_Returns_3m
count,123681.000000,123681.000000,122822.000000,121885.000000,121699.000000
mean,0.009858,1.009858,1.009984,1.010078,0.029450
std,0.098883,0.098883,0.098643,0.098457,0.168574
min,-0.981295,0.018705,0.119565,0.131355,-0.992569
25%,-0.037080,0.962920,0.962979,0.962933,-0.054254
50%,0.011043,1.011043,1.011152,1.011175,0.031580
75%,0.056965,1.056965,1.057059,1.057152,0.113710
max,2.626632,3.626632,3.626632,3.626632,3.892563


,Date,Ticker,Price_Returns_1m,1m,2m,3m,Price_Returns_3m
123844,2021-06-30,DLTR,0.020513,1.020513,0.848564,1.003844,-0.130701
123845,2021-06-30,DRI,0.019270,1.019270,0.976213,1.039478,0.034306
123846,2021-06-30,DPZ,0.095107,1.095107,1.010726,1.148318,0.271020
123847,2021-06-30,NVR,0.017611,1.017611,0.973920,1.065204,0.055694
123848,2021-06-30,NTAP,0.057516,1.057516,1.035882,1.034493,0.133247
123849,2021-06-30,CTXS,0.023275,1.023275,0.928220,0.882374,-0.161900
123850,2021-06-30,DXC,0.026899,1.026899,1.152233,1.052783,0.245681
123851,2021-06-30,AMZN,0.067355,1.067355,0.929530,1.120663,0.111852
123852,2021-06-30,ODFL,-0.043174,0.956826,1.029634,1.072376,0.056485
123853,2021-06-30,HIG,-0.051721,0.948279,0.996058,0.987573,-0.067197


In [9]:
### merge new field with starting csv data so we can export
df = df[['Date','Ticker','Price_Returns_3m']] #limit to just the columns we need to merge
#export_df = csv_data.copy() #copy of starting data
export_df = export_df.merge(df, how='left', on=['Date','Ticker'])

#export_df

## Imputation of Trailing 6 month Price Performance

In [10]:
df = csv_data.copy()

#df = df[df.Ticker == 'IBM']  #Sanity check using one stock


### Limit to just the columns we need
df = df[['Date', 'Ticker','Price_Returns_1m']]

#display(df)


### Shift by __ months and calculate performance consistent with the existing performance columns
### Need the '+1' for 'chaining' the performance together for an accurate performance calc.
df['1m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(0) +1
df['2m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(1) +1
df['3m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(2) +1
df['4m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(3) +1
df['5m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(4) +1
df['6m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(5) +1
# df['7m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(6) +1
# df['8m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(7) +1
# df['9m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(8) +1
# df['10m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(9) +1
# df['11m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(10) +1
# df['12m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(11) +1

### Perform the chained performance calc
df['Price_Returns_6m'] = ( df['1m']*df['2m']*df['3m']*df['4m']*df['5m']*df['6m']  )-1  #*df['7m']*df['8m']*df['9m']*df['10m']*df['11m']*df['12m'] )-1

### Check the output
display(df.describe())

#display(df.head(24))
 
display(df.tail(24))

,Price_Returns_1m,1m,2m,3m,4m,5m,6m,Price_Returns_6m
count,123681.000000,123681.000000,122822.000000,121885.000000,120916.000000,119952.000000,118992.000000,118756.000000
mean,0.009858,1.009858,1.009984,1.010078,1.010007,1.009839,1.009613,0.059918
std,0.098883,0.098883,0.098643,0.098457,0.098337,0.098102,0.097699,0.245836
min,-0.981295,0.018705,0.119565,0.131355,0.156466,0.156466,0.156466,-0.995069
25%,-0.037080,0.962920,0.962979,0.962933,0.962759,0.962617,0.962600,-0.064549
50%,0.011043,1.011043,1.011152,1.011175,1.011041,1.010877,1.010819,0.057609
75%,0.056965,1.056965,1.057059,1.057152,1.057020,1.056801,1.056584,0.177280
max,2.626632,3.626632,3.626632,3.626632,3.626632,3.626632,3.626632,7.727274


,Date,Ticker,Price_Returns_1m,1m,2m,3m,4m,5m,6m,Price_Returns_6m
123844,2021-06-30,DLTR,0.020513,1.020513,0.848564,1.003844,1.165580,0.965965,0.940948,-0.079045
123845,2021-06-30,DRI,0.019270,1.019270,0.976213,1.039478,1.034006,1.174865,0.984209,0.236652
123846,2021-06-30,DPZ,0.095107,1.095107,1.010726,1.148318,1.064175,0.934594,0.966881,0.222253
123847,2021-06-30,NVR,0.017611,1.017611,0.973920,1.065204,1.046678,1.012225,1.089861,0.218988
123848,2021-06-30,NTAP,0.057516,1.057516,1.035882,1.034493,1.160863,0.942203,1.010399,0.252399
123849,2021-06-30,CTXS,0.023275,1.023275,0.928220,0.882374,1.053631,1.002025,1.024673,-0.093331
123850,2021-06-30,DXC,0.026899,1.026899,1.152233,1.052783,1.239493,0.894326,1.095146,0.512233
123851,2021-06-30,AMZN,0.067355,1.067355,0.929530,1.120663,1.000372,0.964672,0.984424,0.056259
123852,2021-06-30,ODFL,-0.043174,0.956826,1.029634,1.072376,1.120390,1.107062,0.993954,0.302479
123853,2021-06-30,HIG,-0.051721,0.948279,0.996058,0.987573,1.317617,1.062890,0.980400,0.280769


In [11]:
### merge new field with starting csv data so we can export
df = df[['Date','Ticker','Price_Returns_6m']] #limit to just the columns we need to merge
#export_df = csv_data.copy() #copy of starting data
export_df = export_df.merge(df, how='left', on=['Date','Ticker'])

#export_df

## Imputation of Trailing 12 month MOVING AVERAGE Price Performance

In [12]:
df = csv_data.copy()

#df = df[df.Ticker == 'IBM']  #Sanity check using one stock


### Limit to just the columns we need
df = df[['Date', 'Ticker','Price_Returns_1m']]

#display(df)


### Shift by __ months and calculate rolling average
df['1m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(0) 
df['2m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(1) 
df['3m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(2) 
df['4m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(3) 
df['5m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(4) 
df['6m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(5) 
df['7m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(6) 
df['8m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(7) 
df['9m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(8) 
df['10m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(9) 
df['11m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(10) 
df['12m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(11) 

### Perform the chained performance calc
df['Moving_Avg_Returns_12m'] = ( df['1m']+df['2m']+df['3m']+df['4m']+df['5m']+df['6m']+df['7m']+df['8m']+df['9m']+df['10m']+df['11m']+df['12m'] )/12

### Check the output
display(df.describe())

#display(df.head(24))
 
display(df.tail(24))

,Price_Returns_1m,1m,2m,3m,4m,5m,6m,7m,8m,9m,10m,11m,12m,Moving_Avg_Returns_12m
count,123681.000000,123681.000000,122822.000000,121885.000000,120916.000000,119952.000000,118992.000000,118031.000000,117077.000000,116126.000000,115175.000000,114235.000000,113297.000000,112980.000000
mean,0.009858,0.009858,0.009984,0.010078,0.010007,0.009839,0.009613,0.009768,0.009649,0.009058,0.009154,0.009320,0.009128,0.009683
std,0.098883,0.098883,0.098643,0.098457,0.098337,0.098102,0.097699,0.097592,0.097540,0.096900,0.096846,0.096770,0.096378,0.026531
min,-0.981295,-0.981295,-0.880435,-0.868645,-0.843534,-0.843534,-0.843534,-0.843534,-0.843534,-0.843534,-0.843534,-0.843534,-0.843534,-0.247500
25%,-0.037080,-0.037080,-0.037021,-0.037067,-0.037241,-0.037383,-0.037400,-0.037217,-0.037396,-0.037574,-0.037446,-0.037203,-0.037326,-0.002840
50%,0.011043,0.011043,0.011152,0.011175,0.011041,0.010877,0.010819,0.011003,0.010895,0.010617,0.010728,0.010982,0.010837,0.010896
75%,0.056965,0.056965,0.057059,0.057152,0.057020,0.056801,0.056584,0.056729,0.056648,0.056072,0.056166,0.056314,0.056159,0.023548
max,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,0.289880


,Date,Ticker,Price_Returns_1m,1m,2m,3m,4m,5m,6m,7m,8m,9m,10m,11m,12m,Moving_Avg_Returns_12m
123844,2021-06-30,DLTR,0.020513,0.020513,-0.151436,0.003844,0.165580,-0.034035,-0.059052,-0.010985,0.209477,-0.011167,-0.051210,0.031280,0.007229,0.010003
123845,2021-06-30,DRI,0.019270,0.019270,-0.023787,0.039478,0.034006,0.174865,-0.015791,0.103167,0.174717,-0.084904,0.162340,0.141897,0.001716,0.060581
123846,2021-06-30,DPZ,0.095107,0.095107,0.010726,0.148318,0.064175,-0.065406,-0.033119,-0.021218,0.037667,-0.110421,0.041994,0.057810,0.046476,0.022676
123847,2021-06-30,NVR,0.017611,0.017611,-0.026080,0.065204,0.046678,0.012225,0.089861,0.020685,0.011153,-0.031846,-0.020445,0.060606,0.206030,0.037640
123848,2021-06-30,NTAP,0.057516,0.057516,0.035882,0.034493,0.160863,-0.057797,0.010399,0.242544,0.214627,0.011528,-0.074910,0.069752,0.009752,0.059554
123849,2021-06-30,CTXS,0.023275,0.023275,-0.071780,-0.117626,0.053631,0.002025,0.024673,0.052668,0.094023,-0.177474,-0.049123,0.017092,-0.034818,-0.015286
123850,2021-06-30,DXC,0.026899,0.026899,0.152233,0.052783,0.239493,-0.105674,0.095146,0.175262,0.189468,0.031933,-0.106607,0.115578,0.085455,0.079331
123851,2021-06-30,AMZN,0.067355,0.067355,-0.070470,0.120663,0.000372,-0.035328,-0.015576,0.028058,0.043440,-0.035754,-0.087579,0.090461,0.147114,0.021063
123852,2021-06-30,ODFL,-0.043174,-0.043174,0.029634,0.072376,0.120390,0.107062,-0.006046,-0.039542,0.068236,0.052233,-0.104496,0.105896,0.078012,0.036715
123853,2021-06-30,HIG,-0.051721,-0.051721,-0.003942,-0.012427,0.317617,0.062890,-0.019600,0.108145,0.155893,0.045035,-0.088752,-0.036508,0.097795,0.047869


In [13]:
### merge new field with starting csv data so we can export
df = df[['Date','Ticker','Moving_Avg_Returns_12m']] #limit to just the columns we need to merge
#export_df = csv_data.copy() #copy of starting data
export_df = export_df.merge(df, how='left', on=['Date','Ticker'])

#export_df

## Imputation of Trailing 6 month MOVING AVERAGE Price Performance

In [14]:
df = csv_data.copy()

#df = df[df.Ticker == 'IBM']  #Sanity check using one stock


### Limit to just the columns we need
df = df[['Date', 'Ticker','Price_Returns_1m']]

#display(df)


### Shift by __ months and calculate rolling average
df['1m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(0) 
df['2m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(1) 
df['3m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(2) 
df['4m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(3) 
df['5m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(4) 
df['6m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(5) 
# df['7m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(6) 
# df['8m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(7) 
# df['9m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(8) 
# df['10m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(9) 
# df['11m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(10) 
# df['12m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(11) 

### Perform the chained performance calc
df['Moving_Avg_Returns_6m'] = ( df['1m']+df['2m']+df['3m']+df['4m']+df['5m']+df['6m']  )/6  #+df['7m']+df['8m']+df['9m']+df['10m']+df['11m']+df['12m'] )/12

### Check the output
display(df.describe())

#display(df.head(24))
 
display(df.tail(24))

,Price_Returns_1m,1m,2m,3m,4m,5m,6m,Moving_Avg_Returns_6m
count,123681.000000,123681.000000,122822.000000,121885.000000,120916.000000,119952.000000,118992.000000,118756.000000
mean,0.009858,0.009858,0.009984,0.010078,0.010007,0.009839,0.009613,0.009781
std,0.098883,0.098883,0.098643,0.098457,0.098337,0.098102,0.097699,0.039408
min,-0.981295,-0.981295,-0.880435,-0.868645,-0.843534,-0.843534,-0.843534,-0.391347
25%,-0.037080,-0.037080,-0.037021,-0.037067,-0.037241,-0.037383,-0.037400,-0.008037
50%,0.011043,0.011043,0.011152,0.011175,0.011041,0.010877,0.010819,0.011604
75%,0.056965,0.056965,0.057059,0.057152,0.057020,0.056801,0.056584,0.029919
max,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,2.626632,0.614689


,Date,Ticker,Price_Returns_1m,1m,2m,3m,4m,5m,6m,Moving_Avg_Returns_6m
123844,2021-06-30,DLTR,0.020513,0.020513,-0.151436,0.003844,0.165580,-0.034035,-0.059052,-0.009098
123845,2021-06-30,DRI,0.019270,0.019270,-0.023787,0.039478,0.034006,0.174865,-0.015791,0.038007
123846,2021-06-30,DPZ,0.095107,0.095107,0.010726,0.148318,0.064175,-0.065406,-0.033119,0.036634
123847,2021-06-30,NVR,0.017611,0.017611,-0.026080,0.065204,0.046678,0.012225,0.089861,0.034250
123848,2021-06-30,NTAP,0.057516,0.057516,0.035882,0.034493,0.160863,-0.057797,0.010399,0.040226
123849,2021-06-30,CTXS,0.023275,0.023275,-0.071780,-0.117626,0.053631,0.002025,0.024673,-0.014300
123850,2021-06-30,DXC,0.026899,0.026899,0.152233,0.052783,0.239493,-0.105674,0.095146,0.076813
123851,2021-06-30,AMZN,0.067355,0.067355,-0.070470,0.120663,0.000372,-0.035328,-0.015576,0.011169
123852,2021-06-30,ODFL,-0.043174,-0.043174,0.029634,0.072376,0.120390,0.107062,-0.006046,0.046707
123853,2021-06-30,HIG,-0.051721,-0.051721,-0.003942,-0.012427,0.317617,0.062890,-0.019600,0.048803


In [15]:
### merge new field with starting csv data so we can export
df = df[['Date','Ticker','Moving_Avg_Returns_6m']] #limit to just the columns we need to merge
#export_df = csv_data.copy() #copy of starting data
export_df = export_df.merge(df, how='left', on=['Date','Ticker'])

#export_df

## Imputation of Trailing 3 month MOVING AVERAGE Price Performance

In [16]:
df = csv_data.copy()

#df = df[df.Ticker == 'IBM']  #Sanity check using one stock


### Limit to just the columns we need
df = df[['Date', 'Ticker','Price_Returns_1m']]

#display(df)


### Shift by __ months and calculate rolling average
df['1m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(0) 
df['2m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(1) 
df['3m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(2) 
# df['4m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(3) 
# df['5m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(4) 
# df['6m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(5) 
# df['7m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(6) 
# df['8m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(7) 
# df['9m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(8) 
# df['10m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(9) 
# df['11m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(10) 
# df['12m'] = df.groupby(['Ticker'], axis=0)['Price_Returns_1m'].shift(11) 

### Perform the chained performance calc
df['Moving_Avg_Returns_3m'] = ( df['1m']+df['2m']+df['3m']  )/3  #+df['4m']+df['5m']+df['6m']  )/6  #+df['7m']+df['8m']+df['9m']+df['10m']+df['11m']+df['12m'] )/12

### Check the output
display(df.describe())

#display(df.head(24))
 
display(df.tail(24))

,Price_Returns_1m,1m,2m,3m,Moving_Avg_Returns_3m
count,123681.000000,123681.000000,122822.000000,121885.000000,121699.000000
mean,0.009858,0.009858,0.009984,0.010078,0.009805
std,0.098883,0.098883,0.098643,0.098457,0.056157
min,-0.981295,-0.981295,-0.880435,-0.868645,-0.603386
25%,-0.037080,-0.037080,-0.037021,-0.037067,-0.016033
50%,0.011043,0.011043,0.011152,0.011175,0.012236
75%,0.056965,0.056965,0.057059,0.057152,0.038412
max,2.626632,2.626632,2.626632,2.626632,0.773609


,Date,Ticker,Price_Returns_1m,1m,2m,3m,Moving_Avg_Returns_3m
123844,2021-06-30,DLTR,0.020513,0.020513,-0.151436,0.003844,-0.042360
123845,2021-06-30,DRI,0.019270,0.019270,-0.023787,0.039478,0.011654
123846,2021-06-30,DPZ,0.095107,0.095107,0.010726,0.148318,0.084717
123847,2021-06-30,NVR,0.017611,0.017611,-0.026080,0.065204,0.018912
123848,2021-06-30,NTAP,0.057516,0.057516,0.035882,0.034493,0.042630
123849,2021-06-30,CTXS,0.023275,0.023275,-0.071780,-0.117626,-0.055377
123850,2021-06-30,DXC,0.026899,0.026899,0.152233,0.052783,0.077305
123851,2021-06-30,AMZN,0.067355,0.067355,-0.070470,0.120663,0.039182
123852,2021-06-30,ODFL,-0.043174,-0.043174,0.029634,0.072376,0.019612
123853,2021-06-30,HIG,-0.051721,-0.051721,-0.003942,-0.012427,-0.022697


In [17]:
### merge new field with starting csv data so we can export
df = df[['Date','Ticker','Moving_Avg_Returns_3m']] #limit to just the columns we need to merge
#export_df = csv_data.copy() #copy of starting data
export_df = export_df.merge(df, how='left', on=['Date','Ticker'])

#export_df

## Look at the Export dataframe before exporting

In [18]:
print(export_df.columns.to_list())

['Date', 'Ticker', 'Name', 'Sector', 'Price', 'Price_Returns_12m', 'Price_Returns_1m', 'Trail_DivYld', 'PB', 'Trail_EV_EBITDA', 'Trail_PE', 'Trail3yrAvg_EPSgro', 'Trail3yrAvg_DPSgro', 'Volatility', 'Debt_to_MktCap', 'NetDebt_EBITDA', 'Trail1yr_EPSgro', 'Trail1yr_DPSgro', 'Price_Returns_3m', 'Price_Returns_6m', 'Moving_Avg_Returns_12m', 'Moving_Avg_Returns_6m', 'Moving_Avg_Returns_3m']


In [19]:
### Look at the export dataframe
column_order = [ 'Date', 'Ticker', 'Name', 'Sector', 'Price', 'Price_Returns_12m', 'Price_Returns_6m', 'Price_Returns_3m', 'Price_Returns_1m',  'Moving_Avg_Returns_12m', 'Moving_Avg_Returns_6m', 'Moving_Avg_Returns_3m','Trail_DivYld', 'PB', 'Trail_EV_EBITDA', 'Trail_PE', 'Trail3yrAvg_EPSgro', 'Trail3yrAvg_DPSgro', 'Volatility', 'Debt_to_MktCap', 'NetDebt_EBITDA', 'Trail1yr_EPSgro', 'Trail1yr_DPSgro',]

export_df = export_df[column_order].copy()
export_df

,Date,Ticker,Name,Sector,Price,Price_Returns_12m,Price_Returns_6m,Price_Returns_3m,Price_Returns_1m,Moving_Avg_Returns_12m,...,PB,Trail_EV_EBITDA,Trail_PE,Trail3yrAvg_EPSgro,Trail3yrAvg_DPSgro,Volatility,Debt_to_MktCap,NetDebt_EBITDA,Trail1yr_EPSgro,Trail1yr_DPSgro
0,2000-12-31,VTSS,MICROCHIP COMMUN,Information Technology,1106.2500,0.054827,NaN,NaN,0.282609,NaN,...,8.6338,36.6065,80.1630,66.4983,0.0000,114.207,0.0721,-0.1106,NaN,NaN
1,2000-12-31,RX,IQVIA INC,Information Technology,27.0000,0.020148,NaN,NaN,-0.035714,NaN,...,75.9729,18.0078,31.3953,NaN,0.0000,56.091,0.0485,0.5787,NaN,NaN
2,2000-12-31,MMC,MARSH & MCLENNAN,Financials,58.5000,0.245283,NaN,NaN,0.016287,NaN,...,6.1773,12.6324,28.5366,-0.0940,14.4714,46.930,0.0834,0.8887,NaN,NaN
3,2000-12-31,MAS,MASCO CORP,Industrials,22.5765,0.037589,NaN,NaN,0.330097,NaN,...,2.9307,NaN,14.8530,2.1774,6.5566,50.488,0.2817,NaN,NaN,NaN
4,2000-12-31,BNI,BURLINGTON NORTH,Industrials,28.3125,0.191993,NaN,NaN,0.123669,NaN,...,1.5223,NaN,11.9462,4.1158,6.2659,44.893,0.5997,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123863,2021-06-30,ROK,ROCKWELL AUTOMAT,Industrials,286.0200,0.365998,0.149864,0.081842,0.084559,0.027373,...,18.2737,24.9142,34.3342,8.7257,8.0849,25.230,0.0715,1.3119,7.669540,4.962575
123864,2021-06-30,KHC,KRAFT HEINZ CO/T,Consumer Staples,40.7800,0.335186,0.199755,0.028890,-0.064464,0.027000,...,0.9918,21.7808,13.5238,-2.9751,-13.8226,24.890,0.5454,7.3184,7.023957,0.001139
123865,2021-06-30,AMT,AMERICAN TOWER C,Real Estate,270.1400,0.066064,0.215391,0.141164,0.062448,0.006499,...,30.9410,24.0730,60.6925,34.7038,19.0551,23.760,0.2987,5.9055,1.444981,17.393419
123866,2021-06-30,DVA,DAVITA INC,Health Care,120.4300,0.521734,0.025809,0.117472,0.002998,0.039449,...,10.3320,10.4995,14.6562,35.1035,0.0000,27.763,0.9469,4.8479,12.186664,0.000000


In [20]:
### export to Google Drive

# export_df.to_csv("/content/gdrive/Shareddrives/Milestone2/SP500_Index_Data_with_Added_Features.csv", index=False)

### Scratchpad Area for examining the export_df

In [21]:
#export_df[['Price_Returns_12m', 'Price_Returns_6m', 'Price_Returns_3m', 'Price_Returns_1m',  'Moving_Avg_Returns_12m', 'Moving_Avg_Returns_6m', 'Moving_Avg_Returns_3m']].corr()

In [22]:
print(export_df.columns.to_list())

['Date', 'Ticker', 'Name', 'Sector', 'Price', 'Price_Returns_12m', 'Price_Returns_6m', 'Price_Returns_3m', 'Price_Returns_1m', 'Moving_Avg_Returns_12m', 'Moving_Avg_Returns_6m', 'Moving_Avg_Returns_3m', 'Trail_DivYld', 'PB', 'Trail_EV_EBITDA', 'Trail_PE', 'Trail3yrAvg_EPSgro', 'Trail3yrAvg_DPSgro', 'Volatility', 'Debt_to_MktCap', 'NetDebt_EBITDA', 'Trail1yr_EPSgro', 'Trail1yr_DPSgro']


In [23]:
### view corr table for price features
export_df[['Price_Returns_12m', 'Price_Returns_6m', 'Price_Returns_3m', 'Price_Returns_1m',  'Moving_Avg_Returns_12m', 'Moving_Avg_Returns_6m', 'Moving_Avg_Returns_3m']].corr().round(2)

,Price_Returns_12m,Price_Returns_6m,Price_Returns_3m,Price_Returns_1m,Moving_Avg_Returns_12m,Moving_Avg_Returns_6m,Moving_Avg_Returns_3m
Price_Returns_12m,1.00,0.59,0.39,0.22,0.93,0.58,0.39
Price_Returns_6m,0.59,1.00,0.66,0.37,0.67,0.96,0.65
Price_Returns_3m,0.39,0.66,1.00,0.53,0.47,0.70,0.98
Price_Returns_1m,0.22,0.37,0.53,1.00,0.28,0.42,0.56
Moving_Avg_Returns_12m,0.93,0.67,0.47,0.28,1.00,0.69,0.48
Moving_Avg_Returns_6m,0.58,0.96,0.70,0.42,0.69,1.00,0.71
Moving_Avg_Returns_3m,0.39,0.65,0.98,0.56,0.48,0.71,1.00


In [24]:
### view corr table for NON-price features
export_df[['Trail_DivYld', 'PB', 'Trail_EV_EBITDA', 'Trail_PE', 'Trail3yrAvg_EPSgro', 'Trail3yrAvg_DPSgro', 'Volatility', 'Debt_to_MktCap', 'NetDebt_EBITDA', 'Trail1yr_EPSgro', 'Trail1yr_DPSgro']].corr().round(2)

,Trail_DivYld,PB,Trail_EV_EBITDA,Trail_PE,Trail3yrAvg_EPSgro,Trail3yrAvg_DPSgro,Volatility,Debt_to_MktCap,NetDebt_EBITDA,Trail1yr_EPSgro,Trail1yr_DPSgro
Trail_DivYld,1.00,0.01,-0.01,-0.02,-0.01,0.13,0.05,0.15,0.03,-0.00,0.19
PB,0.01,1.00,0.00,0.00,0.00,0.01,-0.01,-0.01,-0.00,0.00,0.00
Trail_EV_EBITDA,-0.01,0.00,1.00,0.08,-0.00,-0.01,0.01,-0.00,-0.38,0.00,-0.01
Trail_PE,-0.02,0.00,0.08,1.00,0.00,-0.02,0.03,-0.01,-0.05,-0.00,-0.01
Trail3yrAvg_EPSgro,-0.01,0.00,-0.00,0.00,1.00,0.02,0.01,-0.01,-0.00,0.00,0.05
Trail3yrAvg_DPSgro,0.13,0.01,-0.01,-0.02,0.02,1.00,-0.14,-0.04,-0.01,0.01,0.39
Volatility,0.05,-0.01,0.01,0.03,0.01,-0.14,1.00,0.21,0.02,-0.01,-0.04
Debt_to_MktCap,0.15,-0.01,-0.00,-0.01,-0.01,-0.04,0.21,1.00,0.07,0.00,-0.02
NetDebt_EBITDA,0.03,-0.00,-0.38,-0.05,-0.00,-0.01,0.02,0.07,1.00,0.00,-0.01
Trail1yr_EPSgro,-0.00,0.00,0.00,-0.00,0.00,0.01,-0.01,0.00,0.00,1.00,0.00


In [25]:
### export corr tables to csv, put into excel, and make the colors look nice

# export_df[['Price_Returns_12m', 'Price_Returns_6m', 'Price_Returns_3m', 'Price_Returns_1m',  'Moving_Avg_Returns_12m', 'Moving_Avg_Returns_6m', 'Moving_Avg_Returns_3m']].corr().to_csv("/content/gdrive/Shareddrives/Milestone2/performance_feature_corr.csv", index=True)
# export_df[['Trail_DivYld', 'PB', 'Trail_EV_EBITDA', 'Trail_PE', 'Trail3yrAvg_EPSgro', 'Trail3yrAvg_DPSgro', 'Volatility', 'Debt_to_MktCap', 'NetDebt_EBITDA', 'Trail1yr_EPSgro', 'Trail1yr_DPSgro']].corr().to_csv("/content/gdrive/Shareddrives/Milestone2/fundamental_feature_corr.csv", index=True)